In [2]:
import time
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Marqo
from langchain.vectorstores import LanceDB
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [33]:
import marqo
from langchain_core.load import load
import json

In [70]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

In [53]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [94]:

import requests
from bs4 import BeautifulSoup

In [5]:
def readWebsite2(urls):
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return docs


In [90]:
def loadBookURLs(urls):
    Links = []
    for url in urls:
        reqs = requests.get(url)
        #Only finds links between the list of deaths (gets the main content)
        soup = BeautifulSoup(reqs.text.split("List of deaths")[1], 'html.parser')
        urls = soup.find_all("a")
        for link in urls:
            if "#" not in link.get('href') and "http" not in link.get('href') and "File" not in link.get('href'):
                if link.get('href') not in Links:
                    Links.append(link.get('href'))
    return Links

In [6]:
docs = readWebsite2(["https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Philosopher%27s_Stone", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Chamber_of_Secrets", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Prisoner_of_Azkaban"])

In [81]:
hyperlinks = ["/wiki/J._K._Rowling",
"/wiki/Harry_Potter",
"/wiki/Cloak_of_Invisibility",
"/wiki/Dursley_family",
"/wiki/Petunia_Dursley",
"/wiki/Vernon_Dursley",
"/wiki/Dudley_Dursley",
"/wiki/Cupboard_Under_the_Stairs",
"/wiki/Hogwarts_acceptance_letter",
"/wiki/Owl",
"/wiki/Hogwarts_School_of_Witchcraft_and_Wizardry",
"/wiki/Quidditch",
"/wiki/Magic",
"/wiki/Fall_of_Lord_Voldemort_celebrations",
"/wiki/4_Privet_Drive",
"/wiki/Little_Whinging",
"/wiki/Surrey",
"/wiki/Grunnings",
"/wiki/1_November",
"/wiki/1981",
"/wiki/Potter_family",
"/wiki/Non-magic_people",
"/wiki/Albus_Dumbledore",
"/wiki/Deluminator",
"/wiki/Minerva_McGonagall",
"/wiki/James_Potter_I",
"/wiki/Lily_J._Potter",
"/wiki/Tom_Riddle",
"/wiki/Potter_cottage",
"/wiki/Godric%27s_Hollow",
"/wiki/Rubeus_Hagrid",
"/wiki/Sirius_Black%27s_motorbike",
"/wiki/Albus_Dumbledore%27s_letter_to_Petunia_Dursley",
"/wiki/Sirius_Black",
"/wiki/Vanishment",
"/wiki/Privet_Drive",
"/wiki/Dream",
"/wiki/Harry_Potter%27s_scars",
"/wiki/Arabella_Figg",
"/wiki/Zoo",
"/wiki/Piers_Polkiss",
"/wiki/Piers_Polkiss%27s_mother",
"/wiki/Boa_constrictor_at_the_zoo",
"/wiki/Parseltongue",
"/wiki/Vanishing_Spell",
"/wiki/Dudley_Dursley%27s_gang",
"/wiki/Smeltings_Academy",
"/wiki/Stonewall_High",
"/wiki/Marjorie_Dursley",
"/wiki/Smallest_Bedroom",
"/wiki/Egg",
"/wiki/Toothless_old_man%27s_rowboat",
"/wiki/Hut-on-the-Rock",
"/wiki/Cannon"
"/wiki/Giant",
"/wiki/Harry_Potter%27s_birthday_cake_from_Rubeus_Hagrid",
"/wiki/Keeper_of_Keys_and_Grounds",
"/wiki/Wizardkind",
"/wiki/1_September",
"/wiki/Parchment",
"/wiki/Quill",
"/wiki/Rubeus_Hagrid%27s_letter_to_Albus_Dumbledore",
"/wiki/Headmaster",
"/wiki/Dark_wizard",
"/wiki/Killing_Curse",
"/wiki/Human_to_pig_spell",
"/wiki/Diagon_Alley",
"/wiki/Daily_Prophet",
"/wiki/Wizarding_currency",
"/wiki/Gringotts_Wizarding_Bank",
"/wiki/Philosopher%27s_Stone",
"/wiki/Charm",
"/wiki/London",
"/wiki/British_Ministry_of_Magic",
"/wiki/Minister_for_Magic",
"/wiki/Cornelius_Fudge",
"/wiki/Dragon",
"/wiki/Robes",
"/wiki/Hogwarts_Uniform",
"/wiki/Dragon_hide",
"/wiki/Leaky_Cauldron",
"/wiki/Doris_Crockford",
"/wiki/Dedalus_Diggle",
"/wiki/Quirinus_Quirrell%27s_turban",
"/wiki/Quirinus_Quirrell",
"/wiki/Defence_Against_the_Dark_Arts",
"/wiki/Professor",
"/wiki/Griphook",
"/wiki/Vault_713",
"/wiki/Sickle",
"/wiki/Galleon",
"/wiki/Mine_Cart",
"/wiki/Motion_sickness",
"/wiki/Madam_Malkin%27s_Robes_for_All_Occasions",
"/wiki/Malkin",
"/wiki/Draco_Malfoy",
"/wiki/Broomstick",
"/wiki/Slytherin",
"/wiki/Hufflepuff",
"/wiki/Flying_Ford_Anglia",
"/wiki/The_Burrow"]

In [82]:
documents = []

In [95]:
hyperlinks = loadBookURLs(["https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Philosopher%27s_Stone", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Chamber_of_Secrets", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Prisoner_of_Azkaban", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Goblet_of_Fire", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Order_of_the_Phoenix", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Half-Blood_Prince", "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Deathly_Hallows"])

In [96]:
for link in hyperlinks:
    print(link)
    docs = readWebsite2("https://harrypotter.fandom.com" + link)
    documents.extend(docs)

/wiki/J._K._Rowling
/wiki/Harry_Potter
/wiki/Cloak_of_Invisibility
/wiki/Dursley_family
/wiki/Petunia_Dursley
/wiki/Vernon_Dursley
/wiki/Dudley_Dursley
/wiki/Cupboard_Under_the_Stairs
/wiki/Hogwarts_acceptance_letter
/wiki/Owl
/wiki/Hogwarts_School_of_Witchcraft_and_Wizardry
/wiki/Quidditch
/wiki/Magic
/wiki/Fall_of_Lord_Voldemort_celebrations
/wiki/4_Privet_Drive
/wiki/Little_Whinging
/wiki/Surrey
/wiki/Grunnings
/wiki/1_November
/wiki/1981
/wiki/Potter_family
/wiki/Non-magic_people
/wiki/Albus_Dumbledore
/wiki/Deluminator
/wiki/Minerva_McGonagall
/wiki/James_Potter_I
/wiki/Lily_J._Potter
/wiki/Tom_Riddle
/wiki/Potter_cottage
/wiki/Godric%27s_Hollow
/wiki/Rubeus_Hagrid
/wiki/Sirius_Black%27s_motorbike
/wiki/Albus_Dumbledore%27s_letter_to_Petunia_Dursley
/wiki/Sirius_Black
/wiki/Vanishment
/wiki/Privet_Drive
/wiki/Dream
/wiki/Harry_Potter%27s_scars
/wiki/Arabella_Figg
/wiki/Zoo
/wiki/Piers_Polkiss
/wiki/Piers_Polkiss%27s_mother
/wiki/Boa_constrictor_at_the_zoo
/wiki/Parseltongue
/wiki/

In [97]:
len(documents)

1047

In [8]:
def chunkDocs(doc, size):  
    r_text_splitter = RecursiveCharacterTextSplitter(
        # Set custom chunk size
        chunk_size = size,
        chunk_overlap  = 0,
        separators = [ '\n\n', '\n', '\t\t', '\t', ' ', '']
    )
    split = r_text_splitter.split_documents(doc)
    # splits = r_text_splitter.split_text(doc)
    return split 

In [98]:
splits = chunkDocs(documents, 100)

In [99]:
len(splits)

452943

In [57]:
model_name = "sentence-transformers/all-mpnet-base-v2"
# model_name = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

In [58]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\linew\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [100]:
#takes 6.76 minutes to run with 3 documents and 10 chunks
#takes 38.56 minutes to run with 100 documents and 100 chunks
#takes 291.46 minutes to run with 1000 documents and 100 chunks
start = time.time()
persist_directory = 'data/chroma/'
print("create the vector store")
# Create the vector store
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)
print("persist")
vectordb.persist()
end = time.time()
print((end - start)/60)

create the vector store
persist
291.455828332901


In [101]:
#w/3 documents and 10 chunk it takes ca. 5.40 minutes
#w/100 documents and 100 chunk it takes ca. 39.18 minutes
#w/1000 documents and 100 chunk it takes ca. 301.28 minutes
start = time.time()
from langchain_community.vectorstores import LanceDB
import lancedb
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)
db = LanceDB.from_documents(splits, embeddings)
end = time.time()
print((end - start)/60)

301.2779974102974


In [60]:
llm = Ollama(model="mistral", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [61]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 

{context}

Question: {question}

Helpful Answer:
"""

In [72]:
prompt = PromptTemplate.from_template(template)
chain = RetrievalQA.from_chain_type(
    llm,
    retriever=neo4j_db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [73]:
question = "what model is the flying car?"

In [74]:
start = time.time()
result = chain({"query": question})
result["result"]
end = time.time()
print((end - start)/60)

 I'm sorry, there seems to be missing context in your question as no specific flying car model has been mentioned. Thanks for asking!1.801820441087087


In [ ]:
#chroma w/3 docs, 10 chunk 1.4911341349283853 minutes

#lancedb w/3 docs, 10 chunk 2.9323415637016295 minutes

##Neo4j w/3 docs, 10 chunk 1.801820441087087 minutes

In [71]:
#w/3 documents and 10 chunk it takes ca. 7.49 minutes
start = time.time()
neo4j_db = Neo4jVector.from_documents(
    splits,
    embeddings,
    url="bolt://localhost:7687",
    username="neo4j",
    password="abcd1234",
    database="neo4j",  # neo4j by default
    index_name="harry potter",  # vector by default
    node_label="harry potter",  # Chunk by default
    text_node_property="info",  # text by default
    embedding_node_property="vector",  # embedding by default
    create_id_index=True,  # True by default
)
end = time.time()
print((end - start)/60)

7.486401546001434
